In [1]:
from __future__ import print_function
from __future__ import division
import matplotlib.pyplot as plt
import time
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
print("PyTorch Version: ",torch.__version__)
import sys
sys.path.append("/Users/Lisa/Desktop/Master Thesis/RoofNetXAI/roofnet")
from utils import data as roof
import importlib
import copy

PyTorch Version:  2.0.1


In [2]:
import cv2
import torch
from torch.nn import functional as F
from torchvision import models, transforms


In [3]:
def gradcam(model, image):
    model.eval()

    # Define the transformation for the input image
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # Preprocess the image
    input_image = transform(image).unsqueeze(0)

    # Forward pass
    output = model(input_image)
    _, predicted = torch.max(output.data, 1)
    predicted_class = predicted.item()

    # Perform the gradient calculation
    output[:, predicted_class].backward()

    # Get the gradients from the last convolutional layer
    gradients = model.get_activations_gradient()

    # Global average pooling of gradients
    pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])

    # Get the feature maps from the last convolutional layer
    activations = model.get_activations(input_image).detach()

    # Weight the feature maps by the gradients
    for i in range(activations.shape[1]):
        activations[:, i, :, :] *= pooled_gradients[i]

    # Average the weighted feature maps
    heatmap = torch.mean(activations, dim=1).squeeze()
    heatmap = F.relu(heatmap)

    # Normalize the heatmap
    heatmap /= torch.max(heatmap)

    # Convert the heatmap to a numpy array
    heatmap = heatmap.cpu().numpy()

    return heatmap


In [4]:
#This helper function sets the .requires_grad attribute of the parameters in the model to False when we are feature extracting. 
#By default, when we load a pretrained model all of the parameters have .requires_grad=True, which is fine if we are training from scratch or finetuning. 
#However, if we are feature extracting and only want to compute gradients for the newly initialized layer then we want all of the other parameters to not require gradients. 

def set_parameter_requires_grad(model, feature_extracting): # model is a nn.Module, feature_extracting is a boolean
    if feature_extracting: 
        for param in model.parameters(): # model.parameters() is a generator of tensors
            param.requires_grad = False 

In [9]:
def initialize_model(num_classes, feature_extract, use_pretrained=True):
    model_ft = models.resnet18(pretrained=use_pretrained) # model_ft is a nn.Module
    set_parameter_requires_grad(model_ft, feature_extract) # model_ft is a nn.Module, feature_extract is a boolean
    num_ftrs = model_ft.fc.in_features # num_ftrs is an int which is the number of input features for the last layer
    model_ft.fc = nn.Sequential(nn.Dropout(0.25),nn.Linear(num_ftrs, num_classes)) #Trying with dropout 
    input_size = 224 

    return model_ft, input_size 

In [16]:
# Load the trained model
model = initialize_model(num_classes=128, feature_extract=True)[0]
model.load_state_dict(torch.load('TripleLoss Experiment 64 Train Val'))
model.eval()

# Load the triplet images (anchor, positive, and negative)
anchor_image = cv2.imread('anchor.jpg')
positive_image = cv2.imread('positive.jpg')
negative_image = cv2.imread('negative.jpg')


[ WARN:0@405.024] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('anchor.jpg'): can't open/read file: check file path/integrity
[ WARN:0@405.024] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('positive.jpg'): can't open/read file: check file path/integrity
[ WARN:0@405.024] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('negative.jpg'): can't open/read file: check file path/integrity
